# Medical Chatbot using Generative AI
This notebook recreates the functionality of the [Medical Chatbot project](https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI), allowing PDF input and Generative AI-powered Q&A using LangChain and Pinecone.

In [1]:
# %pip install langchain pinecone-client python-dotenv PyPDF2 matplotlib-inline>=0.1 langchain-google-genai google-generativeai pyreadline3 langchain-pinecone

In [2]:
# %pip install -U  pypdf langchain-community

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from pinecone import Pinecone, PodSpec
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv('.env')
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

pc_api_key = os.getenv("PINECONE_API_KEY")
pc_environment = "us-west1-gcp"
pc = Pinecone(api_key=pc_api_key)


In [15]:

# pdf_path = "Data/paper.pdf"
# pdf_path = "Data/MedicalBook2.pdf"
pdf_path = "Data/derma.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Loaded and split {len(texts)} chunks from PDF.")


Loaded and split 78 chunks from PDF.


In [20]:

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
index_name = "medical-chatbot-gemini"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=768, 
        metric='cosine', 
        spec=PodSpec(environment=pc_environment)
    )

docsearch = PineconeVectorStore.from_documents(
    texts, 
    embeddings, 
    index_name=index_name
)


In [21]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, convert_system_message_to_human=True)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=docsearch.as_retriever()) 


In [18]:
query = "What are the symptoms of hairfall?"
response = qa_chain.run(query)
print("Response:", response)


d:\pyhton_project\daysOfMl\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Response: Hairfall is also known as alopecia. The symptoms of alopecia are hair loss, for example, a well-defined patch of complete hair loss. Hair loss can also be caused by premature entry of hair follicles into the telogen phase.


✅ You're now able to ask medical questions based on the content of your PDF using a Generative AI-powered chatbot!

In [19]:
query = "Who are writer of the book?"
response = qa_chain.run(query)
print("Response:", response)

d:\pyhton_project\daysOfMl\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Response: The writers of the book are Dr. Nicole Yi Zhen Chiang and Professor Julian Verbov.
